In [1514]:
import pandas as pd
import numpy as np
import re

from functools import reduce
import ast

import unicodedata
from sklearn.preprocessing import MultiLabelBinarizer

In [1515]:
pd.set_option("display.max_columns", None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

In [1516]:
#modificado por ruta relatiba Gibhud
#dfbarrios= pd.read_excel('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/distr_barrios_codpostal.xlsx')
#dfi= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionSilvia23122024.csv',index_col=False)

#dfpisos= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionAle29122024.csv')

In [1634]:
#dfbarrios= pd.read_excel('./data/distr_barrios_codpostal.xlsx')
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')
dfi= pd.read_csv('./data/dUnionSilvia23122024.csv',index_col=False)
dfpisos= pd.read_csv('./data/dUnionAle29122024.csv',index_col=False)
#utilizo el fichero pisoscom_limpio que parece que aun tenia los valores de m2_constr
dfpcomlimpio= pd.read_csv('../02-datos-limpios/pisoscom_limpio.csv',index_col=False)
dfisolometros= pd.read_excel('../02-datos-limpios/Indomiosolometros.xlsx')

ParserError: Error tokenizing data. C error: Expected 17 fields in line 7, saw 18


In [1518]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [1519]:
dfpisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4737 entries, 0 to 4736
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   estado              2935 non-null   object 
 1   url                 4736 non-null   object 
 2   distrito            4202 non-null   object 
 3   barrio              4514 non-null   object 
 4   cod_distrito        3992 non-null   float64
 5   cod_barrio          3992 non-null   float64
 6   tipologia           4736 non-null   object 
 7   detalle             2866 non-null   object 
 8   consumo_energia     2805 non-null   object 
 9   referencia          2867 non-null   object 
 10  fecha_publicacion   2805 non-null   object 
 11  m2_utiles           1153 non-null   float64
 12  ano_construccion    1426 non-null   object 
 13  €_comunidad_mes     808 non-null    object 
 14  orientacion         1158 non-null   object 
 15  jardin              204 non-null    object 
 16  aire_a

In [1520]:
dfpisos["cod_barrio"]

0       22.00
1       22.00
2       22.00
3       32.00
4       32.00
        ...  
4732   126.00
4733   126.00
4734   126.00
4735   126.00
4736      NaN
Name: cod_barrio, Length: 4737, dtype: float64

In [1521]:
# Función para extraer desde la palabra clave hasta el final
def extraer_callesincomas(titulo):
    palabras_clave = ["CALLE", "AVENIDA", "PLAZA", "PASEO"]
    titulo = titulo.upper().strip()  # Convertir a mayúsculas y eliminar espacios en blanco al principio y al final
    for palabra in palabras_clave:
        if palabra in titulo:
            return titulo.split(palabra, 1)[1]
    



In [1522]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [1523]:
dfbarrios["barrio"] = dfbarrios["barrio"].astype(object)
dfbarrios["distrito"] = dfbarrios["distrito"].astype(object)
dfbarrios["cod_barrio"] = dfbarrios["cod_barrio"].astype(int)
dfbarrios["cod_distrito"] = dfbarrios["cod_distrito"].astype(int)
dfbarrios["codigo_Postal"] = dfbarrios["codigo_Postal"].astype(object)
dfbarrios.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'distrito']]
dfbarrios.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'barrio']]

In [1524]:
dfi.query("url=='https://www.indomio.es/anuncios/97003125/'")

,Unnamed: 0,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,ciudad,distrito,cod_distrito,barrio,cod_barrio,zona,calle,title2,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,usufructo,nuda_propiedad,derecho_superficie,sup_comercial,tipo_inmueble,armario_empotrado,buhardilla,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,hidromasaje,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,read_all,€_comunidad_mes,acceso_minusvalido,orientacion


In [1525]:
dfpisos.columns


Index(['estado', 'url', 'distrito', 'barrio', 'cod_distrito', 'cod_barrio',
       'tipologia', 'detalle', 'consumo_energia', 'referencia',
       'fecha_publicacion', 'm2_utiles', 'ano_construccion', '€_comunidad_mes',
       'orientacion', 'jardin', 'aire_acondicionado', 'calefaccion', 'cocina',
       'tipo_suelo', 'piscina', 'armarios', 'garaje', 'terraza', 'trastero',
       'ascensor', 'balcon', 'title', 'ubicacion', 'precio', 'habitaciones',
       'banos', 'm2_constr', 'planta', 'EUR/m2', 'fecha_descarga', 'web',
       'interior', 'exterior'],
      dtype='object')

In [1526]:
dfpisos["€_comunidad_mes"].unique()

array([nan, 'Entre 10 y 20 €', 'Entre 20 y 40 €', 'Entre 40 y 60 €',
       '290', '250', 'Entre 80 y 100 €', 'Entre 60 y 80 €',
       'Más de 100 €', '65', '200', '75€', '350  € aparte calefacción',
       '165€ limpieza, pisicina, gimnasio.....', '1.000', 'No tiene',
       '37', '205 incluido ibi que esta prorrateado cada mes', '113,60',
       '130 €', '50€ aprox.', 'Trimestral', '59', '50', '230 + 36',
       '125€ (116,37€-134,66€)', '45,50 €/mes',
       'Gastos comunidad: 170€ ibi: 1.400€', '31€/mes', '99', '99€',
       '250€', '139',
       'Cumunidad 615€ aprox. Incluye calefacción y agua .', '620 euros',
       '684€ incluida agua y calefacción', '358€+consumos',
       'Gastos comunidad 385€ ibi 1.500€', '440€', '364', '400€',
       '545€ incluida calefacción', '600€', '91.26€  ibi 482€', '230',
       'Comunidad: 320€ mensual  / ibi 1.040,96€ anual',
       'Calefaccion incluida', '189', '157 / ibi 924', '308',
       '680€ /ibi: 1.700€', '400', '500',
       'C. : 155 

In [1527]:
#dfp= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/dUnionAle29122024.csv')

In [1528]:
#dfp=dfp[["url","precio","tipologia"]]

In [1529]:
#dfp = dfp[dfp['tipologia'] != 'Finca']

In [1530]:
dfpisos.shape

(4737, 39)

In [1531]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio', 'precio_anterior', 'descuento',
       'EUR/m2', 'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'title2', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_trip

In [1532]:
dfi.query("derecho_superficie.notnull()")[["url","derecho_superficie"]]

,url,derecho_superficie
1281,https://www.indomio.es/anuncios/96405397/,1.00
1282,https://www.indomio.es/anuncios/96405397/,1.00
1283,https://www.indomio.es/anuncios/96405397/,1.00
1284,https://www.indomio.es/anuncios/96405397/,1.00


In [1533]:

# Eliminar filas donde la columna 'derecho_superficie' sea igual a 1.00
dfi = dfi[dfi['derecho_superficie'] != 1.00]




In [1534]:
dfi[["title","title2","read_all"]]

,title,title2,read_all
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",Ático Calle de Núñez de Balboa 2,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test
1,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid","Piso de dos habitaciones buen estado, entreplanta","En Finca de 15 años vendemos 2 viviendas en entreplanta, se dividieron en Obra Nueva, tienen una única Referencia Catastral por lo que se vende como una sola vivienda. Pero son dos alquileres, totalmente independientes, siendo: (Apartamento de 1 dormitorio con baño y un Estudio con cocina independiente y baño) ambos amueblados con 2 puertas de acceso blindadas a descansillo y Contadores de agua y luz para cada vivienda. Con un único recibo de comunidad 123€ y un solo IBI al año.El Estudio se encuentra alquilado de larga duración y el apartamento se está preparando para alquilar de nuevo, pero ahora está vacío por si se quiere vivir en él o buscar inquilino a gusto del comprador. Rentabilidad garantizada. Dan a patio de la finca con derecho de uso de 22 metros, pero no tiene acceso directo.Su distribución es la siguiente: El Apartamento es amplio de un dormitorio, baño con ducha y salón-comedor con cocina integrada mas zona de almacenaje.El Estudio tiene pasillo ancho de entrada a zona diáfana sin dormitorio pero con cocina independiente y baño grande con ducha.Tienen preinstalación de aire acondicionado cada uno, radiadores eléctricos de energía azul. El agua caliente es por paneles solares centralizados, paredes en liso y suelos de gres, puertas en roble, techos de 2,65 lo que les da una gran amplitud. Son luminosos con sol de tarde con grandes ventanales de 2 hojas con rotura de puente térmico y climalit. Persianas térmicas. Puertas Blindadas.La finca es muy tranquila a 200 metros de Plaza de Barajas y del metro a 7 minutos, buenas calidades vivienda en entreplanta pero sin barreras arquitectónicas con ascensor incluso a la entreplanta y al garaje.Garaje Opcional: Se puede adquirir una plaza de garaje en 12.000€ para coche normal o pequeño o una doble en 24.000€ para dos coches, se accede por Montacargas de 5,20 por 2,15.El precio de las plazas de garaje no está incluido en el precio de la vivienda. Es opcional comprar una, dos o ninguna."
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",Casa plurifamiliar Plaza de Italia,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio
3,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",Piso de una habitación Calle de Juan Ramón Jiménez 8,"A tan sólo unos minutos del Paseo de la Castellana, una de las arterias principales de la ciudad, en la misma avenida que el famoso estadio del Real Madrid, el Santiago Bernabeu, y rodeado de un sinfín de tiendas y restaurantes puedes encontrar tu residencia en un piso de lujo, decorado con detalles exquisitos y ayudarte en tu traslado laboral o simplemente para tu descanso.Magnífico piso en venta recién reformado y muy luminoso, ubicado en el corazón del distrito financiero de Madrid.Distribuido en salón comedor bañado con una magnífica luz natural gracias a sus grandes ventanales, cocina equipada a estrenar, recién reformada, amplia habitación con baño completo en suite, armarios empotrados y con detalles que hagan tu estancia más cómoda.El inmueble se entrega tal cual aparece en las fotos, equipado para entrar a vivir.Al estar ubicado en Las Estancias del Eurobuilding, puedes disfrutar de todos los beneficios que esto te

In [1535]:
dfi=dfi.drop(columns="title2")

In [1536]:
#dfpisos = dfpisos.merge(dfp[['url']], on='url', how='left')

In [1537]:
#dfb= pd.read_feather('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/df_distritos.ftr')

In [1538]:
#dfb.columns

In [1539]:
#creo la columna tiene armario
dfpisos['tiene_armario'] = dfpisos['armarios'].apply(lambda x: 1 if pd.notna(x) and x != '' else np.nan)

In [1540]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Convertir los valores de la columna a cadenas de texto y eliminar espacios en blanco
    df[columna] = df[columna].astype(str).str.strip()
    valor = str(valor).strip()
    
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}

In [1541]:
dfbarrios.head()

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n
0,1,Centro,11,1,Palacio,28013,centro,palacio
1,1,Centro,12,2,Embajadores,28012,centro,embajadores
2,1,Centro,13,3,Cortes,28014,centro,cortes
3,1,Centro,14,4,Justicia,28004,centro,justicia
4,1,Centro,15,5,Universidad,28015,centro,universidad


In [1542]:
def procesar_codpostal_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['codigo_Postal'].isna() & df['cod_barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
        #valor=df["codigo_Postal"]
        print(fila['cod_barrio'])
        valores = obtener_valores_por_columna(dfbarrios, 'cod_barrio', fila['cod_barrio']) 
        if valores:    
            print(valores.get('codigo_Postal'))        
            df.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')


In [1543]:
def procesar_distritos_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['distrito'].isna() & df['barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
        valores = obtener_valores_por_columna(dfbarrios, 'barrio', fila['barrio']) 
        if valores:
             df.at[idx, 'cod_barrio'] = valores.get('cod_barrio') 
             df.at[idx, 'cod_distrito'] = valores.get('cod_distrito') 
             df.at[idx, 'distrito'] = valores.get('distrito')
             df.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')






In [1544]:
# Función para normalizar texto (eliminar acentos y convertir a minúsculas)
def normalizar_texto(texto):
 return unidecode(texto).lower()

In [1545]:
#completo distritos nulos con barrio informado
procesar_distritos_nulos(dfpisos, dfbarrios)


In [1546]:
dfpisos.query("title=='Apartamento en venta en Jerónimos'")

,estado,url,distrito,barrio,cod_distrito,cod_barrio,tipologia,detalle,consumo_energia,referencia,fecha_publicacion,m2_utiles,ano_construccion,€_comunidad_mes,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,terraza,trastero,ascensor,balcon,title,ubicacion,precio,habitaciones,banos,m2_constr,planta,EUR/m2,fecha_descarga,web,interior,exterior,tiene_armario,codigo_Postal
72,NaN,https://www.pisos.com/comprar/apartamento-jeronimos28014-45922451620_101800/,Retiro,NaN,NaN,NaN,Apartamento,"Engel & Völkers presenta la oportunidad de diseñar este magnífico piso con terraza y vistas al Parque Retiro, ubicado en la calle Alfonso XII.\n\nEl piso cuenta con reforma ya de hace tiempo, situación perfecta para crear una reforma integral moderna, hecha a su gusto y estándares, creando su vivienda ideal en la calle más emblemática de Jerónimos, con la ventaja de salir de casa y ya estar en frente del Parque Retiro.\n\nUbicado en planta alta, es un piso luminoso; que goza de buena distribución, muy cuadrada y cuenta con terraza, la cual abarca varios huecos a la calle, ideal para dejar una gran zona social y/o dormitorio principal exterior, con preciosas vistas verdes, junto al Parque del Retiro.\nSu configuración actual es un vestíbulo de entrada, cuatro dormitorios y tres baños, cocina, terraza con espectaculares vistas.\nEsta forma permite dejar espacios amplios y ofrece muchísimas posibilidades de renovación.\n\nProbablemente una configuración de reforma moderna será 3-4 dormitorios en suite, gran salón con vistas (o doble salón), cocina y comedor espacioso, aseo de cortesía y si requiere, dormitorio de servicio.\n\nLa finca tiene conserje, techos altos, acceso para personas con movilidad reducida y trastero de gran tamaño. Plaza de Garaje incluida en el precio, más importante, en el corazón de los Jerónimos.\n\nDisponemos de plano.\nEl barrio de Jerónimos, situado en el centro de la capital, es el barrio más prestigioso de Madrid.\nLocalizado en el distrito Retiro, es el centro aristocrático y artístico distinguido por sus hermosas calles y elegantes edificios.\nEs conocido como «The litlle Paris» por sus fachadas históricas, la zona es una combinación de autenticidad y tradición en pleno corazón del Madrid moderno.\nPosee tesoros incalculables, numerosos monumentos, desde la puerta de Alcalá, pasando por las famosas plazas de Cibeles, Neptuno y de la Lealtad, hasta llegar al célebre Museo del Prado.\n\nEs un barrio muy especial de Madrid, muy privilegiado, limita al este con las dos zonas verdes más conocidas de esta cosmopolita: el Real Jardín Botánico y el parque del Retiro, con su lago, jardines exquisitos que lo protegen del bullicio ciudadano.\nSu entorno colinda con el Paseo del Prado y Recoletos y está enmarcado por la milla de oro de las artes y la cultura, rodeado de diversos museos y centro culturales como: el Museo del Prado, Museo Reina Sofía, el Palacio de Velázquez, el Museo Thyssen, el CaixaForum y la Casa de América.\n\nAlgunas instituciones públicas tienen sede aquí, como el bellísimo Palacio de Comunicaciones del Ayuntamiento de Madrid, la Bolsa de Madrid y el Cuartel General de la Armada.\n\nJerónimos dispone de buenas comunicaciones, próximo a las principales arterias de Madrid, así como a la estación central de trenes para trasladarse rápidamente entre ciudades.\nJerónimos, ubicación excepcional, un lugar apetecido por muchos, pero reservado para muy pocos.",No indicado,Ref.: IF77045-I-00YEZ6-W-02V46K,2024-10-01,NaN,Más de 50 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,1,0,Apartamento en venta en Jerónimos,Jerónimos (Distrito Retiro. Madrid Capital) / Jerónimos (Distrito Retiro. Madrid Capital),2800000.00,4.00,4.00,244.00,11475.00,11475.00,2024-10-19,pisos.com,0,0,1.00,NaN
73,NaN,https://www.pisos.com/comprar/apartamento-jeronimos28014-46697950073_101800/,Retiro,NaN,NaN,NaN,Apartamento,"Engel & Völkers presenta este magnífico piso renovado a estrenar en este segundo semestre

In [1547]:
def asignar_zona_distrito_barrio(row):
    if row['zona'] == 'Sanchinarro':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'San Andrés':
        row['distrito'] = 'Villaverde'
        row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
    if row['zona'] == 'Virgen del Cortijo':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'Montecarmelo':
        row['distrito'] = 'Fuencarral-El Pardo'
        row['barrio'] = 'Mirasierra'

    return row

In [1548]:


# Filtrar las filas donde 'distrito' o 'barrio' son nulos
filas_nulas = dfpisos[dfpisos['distrito'].isna() | dfpisos['barrio'].isna()]

for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
    bar = normalizar_texto(fila['ubicacion'].split('(')[0].strip().replace('/', '')).strip()
    print(bar)
    if bar=='salvador':
        bar='elsalvador'
    if bar=='san andres':
        bar='villaverdealto,cascohistoricodevillaverde'
    if bar=='palos de moguer':
        bar='palosdemoguer'
    if bar=='virgen del cortijo-manoteras':
        bar='valdefuentes'  
    if bar=='sanchinarro':
        bar= 'valdefuentes'    
    if bar=='montecarmelo':
        bar='mirasierra'
    if bar=='ambroz':
        bar='cascohistoricodevicalvaro'
    #para los berrocales como es un barrio nuevo, lo creo aunque o exista oficialmente
    if bar=='los berrocales':
        dfpisos.at[idx, 'cod_barrio'] = 195
       
    valores = obtener_valores_por_columna(dfbarrios, 'barrio_n', bar)
    if valores:
      
        if pd.isna(dfpisos.at[idx, 'barrio']): 
            dfpisos.at[idx, 'barrio'] = valores.get('barrio')       
        if pd.isna(dfpisos.at[idx, 'cod_barrio']): 
            dfpisos.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(dfpisos.at[idx, 'cod_distrito']): 
            dfpisos.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(dfpisos.at[idx, 'distrito']): 
            dfpisos.at[idx, 'distrito'] = valores.get('distrito')
        if pd.isna(dfpisos.at[idx, 'codigo_Postal']): 
            dfpisos.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')
        dfpisos.at[idx, 'zona'] = 'bar'

jeronimos
jeronimos
carmenes

concepcion
jeronimos

palos de moguer
pau de carabanchel
pau de carabanchel
salvador

san andres
san andres
san andres
san andres
san andres
san andres
salvador
salvador
salvador
salvador
san andres
concepcion
san andres

ambroz

concepcion
concepcion
jeronimos
jeronimos
carmenes
carmenes
san andres
san andres
san andres
palos de moguer
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
concepcion
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
jeronimos
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
carmenes
c

In [1549]:
dfb.query("barrio=='Palos de Moguer'")

,cod_distrito,distrito,cod_barrio,barrio,distrito_n,barrio_n


In [1550]:
#pongo los valores a pelo de cod_barrio y distrito que no tengo
dfpisos.query("referencia=='Ref.: 4510732-21249875'")

,estado,url,distrito,barrio,cod_distrito,cod_barrio,tipologia,detalle,consumo_energia,referencia,fecha_publicacion,m2_utiles,ano_construccion,€_comunidad_mes,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,terraza,trastero,ascensor,balcon,title,ubicacion,precio,habitaciones,banos,m2_constr,planta,EUR/m2,fecha_descarga,web,interior,exterior,tiene_armario,codigo_Postal,zona
272,A estrenar,https://www.pisos.com/comprar/atico-los_berrocales28031-4666816647_109700/,Vicálvaro,NaN,NaN,NaN,Ático,"Portal 2, A C. Ático en una finca de obra nueva de 157 m2 de superficie construida y emplazado en la zona de Madrid Capital, posicionado en planta ático. Está dispuesto en 3 habitaciones, 2 baños completos y terraza. Cuenta como extras garaje y trastero.",En trámite,Ref.: 4510732-21249875,2024-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,Ático en venta en los Berrocales,/ / /,498000.00,3.00,3.00,2.00,157.00,3171.00,2024-10-19,pisos.com,0,0,NaN,NaN,NaN


In [1551]:
# Filtrar las filas donde 'referencia' es igual a 'Ref.: 4510732-21249875' 
 #filas_referencia = dfpisos.query("referencia == 'Ref.: 4510732-21249875'")
#  # Asignar el valor 195 a la columna 'cod_barrio' en las filas filtradas
dfpisos.loc[dfpisos.query("referencia == 'Ref.: 4510732-21249875'").index, 'cod_barrio']=195


In [1552]:
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77095-LCM30872-1'").index, 'cod_barrio']=155
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77095-LCM30872-1'").index, 'barrio']='San Pascual'
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77095-LCM30872-1'").index, 'codigo_Postal']=28027

In [1553]:
dfpisos.loc[dfpisos.query("referencia == 'Ref.: GM17-200378'").index, 'cod_barrio']=191
dfpisos.loc[dfpisos.query("referencia == 'Ref.: IF77045-I-00YN6P-W-02X311'").index, 'cod_barrio']=166
dfpisos.loc[dfpisos.query("referencia == 'Ref.: SA283-SIM1201071'").index, 'cod_barrio']=201

In [1554]:
dfbarrios.tail(20)

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n
112,18,Villa de Vallecas,181,1,Casco Histórico de Vallecas,28031,villadevallecas,cascohistoricodevallecas
113,18,Villa de Vallecas,182,2,Santa Eugenia,28031,villadevallecas,santaeugenia
114,18,Villa de Vallecas,183,3,Ensanche de Vallecas,28031,villadevallecas,ensanchedevallecas
115,19,Vicálvaro,191,1,Casco Histórico de Vicálvaro,28032,vicalvaro,cascohistoricodevicalvaro
116,19,Vicálvaro,192,2,Valdebernardo,28032,vicalvaro,valdebernardo
117,19,Vicálvaro,193,3,Valderrivas,28032,vicalvaro,valderrivas
118,19,Vicálvaro,194,4,El Cañaveral,28032,vicalvaro,elcanaveral
119,20,San Blas-Canillejas,201,1,Simancas,28037,sanblas-canillejas,simancas
120,20,San Blas-Canillejas,202,2,Hellín,28037,sanblas-canillejas,hellin
121,20,San Blas-Canillejas,203,3,Amposta,28037,sanblas-canillejas,amposta


In [1555]:
dfbarrios["distrito"] = dfbarrios["distrito"].apply(lambda x: x.strip())


In [1556]:
#teniendo codigo de barrio, pongo el resto de columnas sino tienen valor
# Filtrar las filas donde 'distrito' o 'barrio' son nulos
filas_nulas = dfpisos[dfpisos['distrito'].isna() | dfpisos['barrio'].isna()  | dfpisos['cod_distrito'].isna()]

for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
    bar = (fila['cod_barrio'])
    print(bar)
    fila = dfbarrios.loc[dfbarrios['cod_barrio'] == bar]
    if not fila.empty:
       valores=fila.iloc[0].to_dict()
       print(valores)
       dfpisos.at[idx, 'barrio'] = valores.get('barrio')
       dfpisos.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
       dfpisos.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')
       dfpisos.at[idx, 'distrito'] = valores.get('distrito')


nan
166.0
{'cod_distrito': 16, 'distrito': 'Hortaleza', 'cod_barrio': 166, 'cod_barrio_2': 6, 'barrio': 'Piovera', 'codigo_Postal': 28033, 'distrito_n': 'hortaleza', 'barrio_n': 'piovera'}
195.0
nan
nan
nan
201.0
{'cod_distrito': 20, 'distrito': 'San Blas-Canillejas', 'cod_barrio': 201, 'cod_barrio_2': 1, 'barrio': 'Simancas', 'codigo_Postal': 28037, 'distrito_n': 'sanblas-canillejas', 'barrio_n': 'simancas'}
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
155.0
{'cod_distrito': 15, 'distrito': 'Ciudad Lineal', 'cod_barrio': 155, 'cod_barrio_2': 5, 'barrio': 'San Pascual', 'codigo_Postal': 28027, 'distrito_n': 'ciudadlineal', 'barrio_n': 'sanpascual'}
191.0
{'cod_distrito': 19, 'distrito': 'Vicálvaro', 'cod_barrio': 191, 'cod_barrio_2': 1, 'barrio': 'Casco Histórico de Vicálvaro', 'codigo_Postal': 28032, 'distrito_n': 'vicalvaro', 'barrio_n': 'cascohistoricodevicalvaro'}
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
195.0
195.0
195.0
195.0
195.0
19

In [1557]:
dfi.shape,dfpisos.shape

((3146, 75), (4737, 42))

In [1558]:
dfpisos.columns

Index(['estado', 'url', 'distrito', 'barrio', 'cod_distrito', 'cod_barrio',
       'tipologia', 'detalle', 'consumo_energia', 'referencia',
       'fecha_publicacion', 'm2_utiles', 'ano_construccion', '€_comunidad_mes',
       'orientacion', 'jardin', 'aire_acondicionado', 'calefaccion', 'cocina',
       'tipo_suelo', 'piscina', 'armarios', 'garaje', 'terraza', 'trastero',
       'ascensor', 'balcon', 'title', 'ubicacion', 'precio', 'habitaciones',
       'banos', 'm2_constr', 'planta', 'EUR/m2', 'fecha_descarga', 'web',
       'interior', 'exterior', 'tiene_armario', 'codigo_Postal', 'zona'],
      dtype='object')

In [1559]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio', 'precio_anterior', 'descuento',
       'EUR/m2', 'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'armario_empotrado', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/

In [1560]:
dfi["web"]="Indomio.com"
dfi["fecha_descarga"]="2024-11-20"

In [1561]:
nuevos_nombres ={'armario_empotrado': 'tiene_armario','read_all':'detalle'
 } 
dfi = dfi.rename(columns=nuevos_nombres)

In [1562]:
nuevos_nombres ={'consumo_energia': 'certificado_energetico'
 } 
dfpisos = dfpisos.rename(columns=nuevos_nombres)


In [1563]:
dfi['certificado_energetico'] = dfi['letrace'].apply(lambda x: 'Disponible' if pd.notna(x) and x != '' else 'No indicado')


In [1564]:

# Supongamos que ya tienes tu DataFrame dfpisos
# Eliminar las filas donde 'tipologia' es 'finca'
dfpisos = dfpisos[dfpisos['tipologia'] != 'Finca']


In [1565]:
dfi[["detalle","title"]]

,detalle,title
0,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid"
1,"En Finca de 15 años vendemos 2 viviendas en entreplanta, se dividieron en Obra Nueva, tienen una única Referencia Catastral por lo que se vende como una sola vivienda. Pero son dos alquileres, totalmente independientes, siendo: (Apartamento de 1 dormitorio con baño y un Estudio con cocina independiente y baño) ambos amueblados con 2 puertas de acceso blindadas a descansillo y Contadores de agua y luz para cada vivienda. Con un único recibo de comunidad 123€ y un solo IBI al año.El Estudio se encuentra alquilado de larga duración y el apartamento se está preparando para alquilar de nuevo, pero ahora está vacío por si se quiere vivir en él o buscar inquilino a gusto del comprador. Rentabilidad garantizada. Dan a patio de la finca con derecho de uso de 22 metros, pero no tiene acceso directo.Su distribución es la siguiente: El Apartamento es amplio de un dormitorio, baño con ducha y salón-comedor con cocina integrada mas zona de almacenaje.El Estudio tiene pasillo ancho de entrada a zona diáfana sin dormitorio pero con cocina independiente y baño grande con ducha.Tienen preinstalación de aire acondicionado cada uno, radiadores eléctricos de energía azul. El agua caliente es por paneles solares centralizados, paredes en liso y suelos de gres, puertas en roble, techos de 2,65 lo que les da una gran amplitud. Son luminosos con sol de tarde con grandes ventanales de 2 hojas con rotura de puente térmico y climalit. Persianas térmicas. Puertas Blindadas.La finca es muy tranquila a 200 metros de Plaza de Barajas y del metro a 7 minutos, buenas calidades vivienda en entreplanta pero sin barreras arquitectónicas con ascensor incluso a la entreplanta y al garaje.Garaje Opcional: Se puede adquirir una plaza de garaje en 12.000€ para coche normal o pequeño o una doble en 24.000€ para dos coches, se accede por Montacargas de 5,20 por 2,15.El precio de las plazas de garaje no está incluido en el precio de la vivienda. Es opcional comprar una, dos o ninguna.","Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid"
2,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid"
3,"A tan sólo unos minutos del Paseo de la Castellana, una de las arterias principales de la ciudad, en la misma avenida que el famoso estadio del Real Madrid, el Santiago Bernabeu, y rodeado de un sinfín de tiendas y restaurantes puedes encontrar tu residencia en un piso de lujo, decorado con detalles exquisitos y ayudarte en tu traslado laboral o simplemente para tu descanso.Magnífico piso en venta recién reformado y muy luminoso, ubicado en el corazón del distrito financiero de Madrid.Distribuido en salón comedor bañado con una magnífica luz natural gracias a sus grandes ventanales, cocina equipada a estrenar, recién reformada, amplia habitación con baño completo en suite, armarios empotrados y con detalles que hagan tu estancia más cómoda.El inmueble se entrega tal cual aparece en las fotos, equipado para entrar a vivir.Al estar ubicado en Las Estancias del Eurobuilding, puedes disfrutar de todos los beneficios que esto te aporta, se accede al hotel directamente sin tener que salir a la calle. Considerado el mejor hotel de ocio y negocio en Europa gracias a sus instalaciones, servicio e innovación tecnológica.Dispone también de uno de los mejores spas y Fitness Center de la cap

In [1566]:
dfpisos.query("habitaciones==12")[['url','habitaciones','title']]

,url,habitaciones,title
486,https://www.pisos.com/comprar/chalet-fuencarral_el_pardo_el_pardo28023-31747499198_102100/,12.00,Chalet en venta en La Florida
1614,https://www.pisos.com/comprar/piso-chamartin_hispanoamerica28036-44174087083_526913/,12.00,"Piso en venta en Paseo de la Habana, cerca de Avenida de Concha Espina"
3770,https://www.pisos.com/comprar/piso-salamanca_castellana28006-35920887428_106700/,12.00,Piso en venta en Castellana


In [1567]:
# Convertir la columna 'habitaciones' a numérico, reemplazando valores no convertibles por NaN
dfpisos['habitaciones'] = pd.to_numeric(dfpisos['habitaciones'], errors='coerce')

# Asignar valores a 'dormitorios' cuando 'habitaciones' es menor que 13
dfpisos.loc[dfpisos['habitaciones'] < 13, 'dormitorios'] = dfpisos['habitaciones']

# Eliminar la columna 'habitaciones'
dfpisos = dfpisos.drop(columns=['habitaciones'])



In [1568]:
#dfpisos["distrito"] = dfpisos["distrito"].apply(lambda x: x.strip())

In [1569]:
dfi.columns

Index(['Unnamed: 0', 'title', 'url', 'precio', 'precio_anterior', 'descuento',
       'EUR/m2', 'm2_constr', 'ciudad', 'distrito', 'cod_distrito', 'barrio',
       'cod_barrio', 'zona', 'calle', 'consumoce_ano', 'letrace',
       'emisiones_co2', 'tipologia', 'propiedad_completa', 'estado',
       'amueblado', 'planta', 'habitaciones', 'dormitorios', 'banos', 'balcon',
       'terraza', 'garaje', 'trastero', 'ascensor', 'porteria', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'inmueble_ingresos', 'alquiler_opcion_a_compra', 'disponibilidad',
       'usufructo', 'nuda_propiedad', 'derecho_superficie', 'sup_comercial',
       'tipo_inmueble', 'tiene_armario', 'buhardilla', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/meta

In [1570]:
# Eliminar la primera columna 
dfi = dfi.iloc[:,1:]

In [1571]:
dfpisos.query("precio.isnull()")

,estado,url,distrito,barrio,cod_distrito,cod_barrio,tipologia,detalle,certificado_energetico,referencia,fecha_publicacion,m2_utiles,ano_construccion,€_comunidad_mes,orientacion,jardin,aire_acondicionado,calefaccion,cocina,tipo_suelo,piscina,armarios,garaje,terraza,trastero,ascensor,balcon,title,ubicacion,precio,banos,m2_constr,planta,EUR/m2,fecha_descarga,web,interior,exterior,tiene_armario,codigo_Postal,zona,dormitorios
135,A estrenar,https://www.pisos.com/comprar/atico-arcos28032-4535639357_109700/,San Blas-Canillejas,Arcos,20.00,204.00,Ático,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,"Ático en venta en Calle de Aquitania, s/n",Arcos (Distrito San Blas. Madrid Capital),NaN,1.00,69.00,NaN,NaN,2024-08-19,pisos.com,0,0,NaN,28037.00,NaN,1.00
731,En buen estado,https://www.pisos.com/comprar/estudio-casco_historico_de_barajas-46742905658_521139/,Barajas,Casco Histórico de Barajas,21.00,213.00,Estudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,"Estudio en venta en Calle de Camarillas, 4",Casco Histórico de Barajas (Distrito Barajas. Madrid Capital),NaN,1.00,50.00,NaN,NaN,2024-08-19,pisos.com,0,0,NaN,NaN,NaN,1.00
882,En buen estado,https://www.pisos.com/comprar/piso-adelfas28007-40845633641_526615/,Retiro,Adelfas,3.00,32.00,Piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,"Piso en venta en Calle Doctor Esquerdo, cerca de Paseo del Marqués de Zafra",Adelfas (Distrito Retiro. Madrid Capital),NaN,1.00,55.00,3.00,NaN,2024-08-19,pisos.com,0,0,NaN,NaN,NaN,1.00
3461,A estrenar,https://www.pisos.com/comprar/piso-puerta_bonita28025-4281208855_109700/,Carabanchel,Puerta Bonita,11.00,115.00,Piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,"Piso en venta en Calle del Real Betis, 11",Puerta Bonita (Distrito Carabanchel. Madrid Capital),NaN,2.00,104.00,1.00,NaN,2024-08-19,pisos.com,0,0,NaN,NaN,NaN,3.00
4736,NaN,NaN,Salamanca,Lista,4.00,45.00,NaN,NaN,En trámite,Ref.: SA2325-AR02879/7288,2024-10-24,83.00,Más de 50 años,Más de 100 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,Lista (Distrito Salamanca. Madrid Capital) / /,NaN,2.00,92.00,3.00,9701.00,2024-10-19,pisos.com,0,0,NaN,28006.00,bar,2.00


In [1572]:
# Eliminar las filas donde el precio es nulo
dfpisos = dfpisos.dropna(subset=['precio'])

In [1573]:
dfpisos.shape,dfi.shape

((4728, 42), (3146, 77))

In [1574]:
#dfi = dfi.drop(columns=['ciudad'])
#dfpisos = dfpisos.drop(columns=['ciudad'])

In [1575]:
# Reemplazar "Sí" por 1 y "No" por 0 
dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})
dfpisos["zona"]=dfpisos["ubicacion"]

# Actualizar 'nuda_propiedad' con el valor de 'usufructo' cuando 'usufructo' es 1 y 'nuda_propiedad' es NaN
dfi.loc[(dfi['usufructo'] == 1) & (dfi['nuda_propiedad'].isna()), 'nuda_propiedad'] = dfi['usufructo']

# Definir las columnas a eliminar
columnas_a_eliminar = ['habitaciones','derecho_superficie','usufructo','buhardilla','hidromasaje', 'ciudad' ,'instalacion_tv_individual','instalacion_tv_centralizada']

# Eliminar las columnas
dfi = dfi.drop(columns=columnas_a_eliminar)


# Definir las columnas a eliminar
columnas_a_eliminar = ['ubicacion']

# Eliminar las columnas
dfpisos = dfpisos.drop(columns=columnas_a_eliminar)



C:\Users\extas\AppData\Local\Temp\ipykernel_7680\2380896923.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
C:\Users\extas\AppData\Local\Temp\ipykernel_7680\2380896923.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})


In [1576]:
dfpisos.columns

Index(['estado', 'url', 'distrito', 'barrio', 'cod_distrito', 'cod_barrio',
       'tipologia', 'detalle', 'certificado_energetico', 'referencia',
       'fecha_publicacion', 'm2_utiles', 'ano_construccion', '€_comunidad_mes',
       'orientacion', 'jardin', 'aire_acondicionado', 'calefaccion', 'cocina',
       'tipo_suelo', 'piscina', 'armarios', 'garaje', 'terraza', 'trastero',
       'ascensor', 'balcon', 'title', 'precio', 'banos', 'm2_constr', 'planta',
       'EUR/m2', 'fecha_descarga', 'web', 'interior', 'exterior',
       'tiene_armario', 'codigo_Postal', 'zona', 'dormitorios'],
      dtype='object')

In [1577]:
dfi.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letrace', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'ano_construccion', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carp

que tengo en dfi que no tengo en dfpisos

['precio_anterior', 'descuento', 'zona', 'calle', 'consumoce_ano', 
 'letrace', 'emisiones_co2', 'propiedad_completa', 'amueblado', 
 'dormitorios', 'porteria', 'plantas_edificio', 'inmueble_ingresos', 
 'alquiler_opcion_a_compra', 'disponibilidad', 'usufructo', 'nuda_propiedad',
 'derecho_superficie', 'sup_comercial', 'tiene_armario', 'buhardilla', 
 'cancha_tenis', 'carpinteria_exterior_doble_vidrio/pvc', 
 'carpinteria_exterior_doble_vidrio/madera', 
 'carpinteria_exterior_doble_vidrio/metal', 
 'carpinteria_exterior_triple_vidrio/madera', 
 'carpinteria_exterior_triple_vidrio/metal', 
 'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera', 
 'carpinteria_exterior_vidrio/metal', 'chimenea', 'fibra_optica', 
 'hidromasaje', 'instalacion_tv_centralizada', 'instalacion_tv_individual', 
 'interior_y_exterior', 'porton_electrico', 'puerta_blindada', 'alarma', 
 'videoportero', 'lujosa', 'acceso_minusvalido']

 que tengo en dfpisos que no tengo en dfi

 [ 'referencia', 'fecha_publicacion',  'm2_utiles', 'tipo_suelo' ]



In [1578]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3146 entries, 0 to 3149
Data columns (total 69 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      3146 non-null   object 
 1   url                                        3146 non-null   object 
 2   precio                                     3146 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  55 non-null     object 
 5   EUR/m2                                     3140 non-null   float64
 6   m2_constr                                  3140 non-null   float64
 7   distrito                                   3044 non-null   object 
 8   cod_distrito                               3146 non-null   int64  
 9   barrio                                     3044 non-null   object 
 10  cod_barrio                   

In [1579]:
dfi["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
No indicado    1835
Disponible     1311
Name: count, dtype: int64

In [1580]:
# Asegurarse de que ambos dataframes tengan las mismas columnas, rellenando con NaN 
dfpisos = dfpisos.reindex(columns=dfi.columns, fill_value=None) 
dfi = dfi.reindex(columns=dfpisos.columns, fill_value=None) 
# # Sumar las filas 

#dfi['precio'] = dfi['precio'].str.replace('.', '').str.replace(',', '')
numeric_columns = ['cod_distrito','cod_barrio','trastero','dormitorios','EUR/m2'] # Añade aquí todas las columnas numéricas 
dfi[numeric_columns] = dfi[numeric_columns].astype(float) 
dfpisos[numeric_columns] = dfpisos[numeric_columns].astype(float)

non_numeric_columns = ['tipologia','detalle','precio','banos','piscina','garaje','planta','porteria','amueblado','ano_construccion','title', 'url', 'distrito', 'estado','descuento','letrace','emisiones_co2','consumoce_ano','tipo_inmueble','zona','calle'] # Añade aquí todas las columnas no numéricas
dfi[non_numeric_columns] = dfi[non_numeric_columns].astype(str) 
dfpisos[non_numeric_columns] = dfpisos[non_numeric_columns].astype(str)

# # Sumar las filas 
df_union = pd.concat([dfi, dfpisos], ignore_index=True)
#df_union = dfi.add(dfpisos, fill_value=None)

In [1581]:
df_union.shape

(7874, 69)

In [1582]:
df_union.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letrace', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'ano_construccion', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carp

In [1583]:
df_union=df_union.rename(columns={'€_comunidad_mes': 'gastos_comunidad'})

In [1584]:

# Reemplazar los valores según las indicaciones
df_union['piscina'] = df_union['piscina'].replace({
    '-nan': 'NaN',
    'NaN': 'NaN',
    '-Comunitaria': 'Comunitaria',
    '-Propia': 'Privada',
    '-Con salorium': 'Comunitaria',
    '-1 piscina de adultos y 1 piscina de niños': 'Comunitaria'
})



In [1585]:
df_union['tiene_jardin'] = df_union['jardin'].apply(lambda x: 1 if pd.notnull(x) and x != 'Sin jardín' else (0 if x == 'Sin jardín' else None))

In [1586]:
df_union['jardin'] = df_union['jardin'].replace({
    'Jardín privado': 'Privado',
    'Jardín comunitario': 'Comunitario',
    'Jardín privado y comunitario': 'Privado'
})


In [1587]:
#Reemplazar todas las tipologías que comienzan con "Casa" por "Casa" 
df_union['tipologia'] = df_union['tipologia'].replace(r'^Casa.*', 'Casa', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Apartamento.*', 'Apartamento', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Caserío.*', 'Casa', regex=True)

In [1588]:

# Eliminar las filas donde 'precio' es nulo 
df_union = df_union.dropna(subset=['precio'])

In [1589]:
df_union.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'consumoce_ano', 'letrace', 'emisiones_co2', 'tipologia',
       'propiedad_completa', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'balcon', 'terraza', 'garaje', 'trastero', 'ascensor',
       'porteria', 'calefaccion', 'ano_construccion', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'sup_comercial', 'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carp

In [1590]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7874 entries, 0 to 7873
Data columns (total 70 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      7874 non-null   object 
 1   url                                        7874 non-null   object 
 2   precio                                     7874 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  7874 non-null   object 
 5   EUR/m2                                     7856 non-null   float64
 6   m2_constr                                  7861 non-null   float64
 7   distrito                                   7874 non-null   object 
 8   cod_distrito                               7766 non-null   float64
 9   barrio                                     7664 non-null   object 
 10  cod_barrio              

In [1591]:
df_union["descripcion_larga"]=df_union["detalle"]
df_union=df_union.drop(columns="detalle")

In [1592]:
# Aplicar la función solo si 'calle' es nulo
df_union['calle'] = df_union.apply(
    lambda row: extraer_callesincomas(row['titulo']) if pd.isnull(row['calle']) else row['calle'],
    axis=1
)


In [1593]:
tit='Piso en venta en Cármenes'
df_union.loc[df_union["title"] == tit, ['cod_barrio', 'barrio', 'cod_distrito', 'distrito', 'codigo_Postal']] = [101,'Los Cármenes',10,'Latina',28047]
tit='Apartamento en venta en Calle de los Chisperos'
df_union.loc[df_union["title"] == tit, ['cod_barrio', 'barrio', 'cod_distrito', 'distrito', 'codigo_Postal']] = [101,'Los Cármenes',10,'Latina',28047]

In [1594]:
dfcalles= pd.read_excel('../02-datos-limpios/CALLEJERO_VIGENTE_NUMERACIONES_202501.xlsx')

In [1595]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [1596]:


#dfcalles.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfcalles.loc[:, 'Nombre del distrito']]
#dfcalles.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfcalles.loc[:, 'Nombre del barrio']]

# Normalizar y limpiar la columna 'Nombre de la vía'
dfcalles['calle_n'] = dfcalles['Nombre de la vía'].astype(str).str.strip().str.replace("Calle", "").str.replace("Avenida", "").str.replace("Plaza", "").str.strip().apply(normalize_string).str.replace(" ", "")

def clasificar_distrito_y_barrio_calle(df, dfcalles):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Verificar si la columna "barrio" es nula
        if pd.isna(fila["barrio"]):
            # Recorrer las columnas en orden de prioridad: calle
            for columna in ['title']:
                valor = fila[columna]
                
                # Verificamos que el valor no sea nulo
                if pd.notna(valor):
                    valor_normalizado = normalize_string(valor.strip().replace("Calle", "").replace("Avenida", "").replace("Plaza", "").replace(" ", ""))
                    
                    # Verificamos si el valor normalizado contiene el nombre de una calle
                    calle_encontrada = next((calle for calle in dfcalles["calle_n"] if calle in valor_normalizado), None)
                    
                    if calle_encontrada:
                        # Buscar los valores cod_barrio, distrito y cod_distrito en dfcalles
                        result = dfcalles.query(f"calle_n == '{calle_encontrada}'")
                        if not result.empty:
                            barrio = result["Nombre del barrio"].values[0]
                            cod_distrito = result["Codigo de distrito"].values[0]
                            cod_barrio = result["Codigo de barrio"].values[0]
                            codigo_Postal = result["Codigo postal"].values[0]
                            distrito_result = result["Nombre del distrito"].values[0]
                            
                            # Asignar los valores encontrados a las columnas correspondientes
                            df.at[idx, "barrio"] = barrio
                            df.at[idx, "cod_barrio"] = cod_barrio
                            df.at[idx, "cod_distrito"] = cod_distrito
                            df.at[idx, "distrito"] = distrito_result
                            df.at[idx, "codigo_Postal"] = codigo_Postal
                        
    return df




In [1597]:
# lamar a la función pasándole el dataframe y el dataframe de referencia

clasificar_distrito_y_barrio_calle(df_union, dfcalles)

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",https://www.indomio.es/anuncios/75404850/,2.000.000,NaN,nan,20.00,100.00,Salamanca,4.00,Recoletos,41.00,Recoletos,calle de nunez de balboa 2,"≥ 3,51",F,D,Ático,1.00,Para reformar,Sólo cocina amueblada,1,1.00,1,1.00,1.00,2 en garaje privado,1.00,1.00,Portero media jornada,Central,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,1.00,1,0,0.00,1,0.00,1.00,0.00,1.00,1.00,100,1.00,NaN,Indomio.com,2024-11-20,Disponible,NaN,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test,NaN
1,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid",https://www.indomio.es/anuncios/79178265/,287.000,NaN,nan,3.46,83.00,Barajas,21.00,Casco Histórico de Barajas,213.00,Casco Histórico de Barajas,nan,desconocido,nan,No indicado,Piso,1.00,Bueno / Habitable,Sí,Entreplanta,2.00,2,0.00,1.00,1 en aparcamiento/garaje público,0.00,1.00,nan,"Individual, por radiadores, alimentación eléctrica",2009.0,2.00,Preinstalación,NaN,1.00,NaN,NaN,NaN,NaN,medio,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Cocina abierta,1.00,0,1,0.00,0,0.00,1.00,0.00,0.00,0.00,123,1.00,NaN,Indomio.com,2024-11-20,No indicado,NaN,"En Finca de 15 años vendemos 2 viviendas en entreplanta, se dividieron en Obra Nueva, tienen una única Referencia Catastral por lo que se vende como una sola vivienda. Pero son dos alquileres, totalmente independientes, siendo: (Apartamento de 1 dormitorio con baño y un Estudio con cocina independiente y baño) ambos amueblados con 2 puertas de acceso blindadas a descansillo y Contadores de agua y luz para cada vivienda. Con un único recibo de comunidad 123€ y un solo IBI al año.El Estudio se encuentra alquilado de larga duración y el apartamento se está preparando para alquilar de nuevo, pero ahora está vacío por si se quiere vivir en él o buscar inquilino a gusto del comprador. Rentabilidad garantizada. Dan a patio de la finca con derecho de uso de 22 metros, pero no tiene acceso directo.Su distribución es la siguiente: El Apartamento es amplio de un dormitorio, baño con ducha y salón-comedor con cocina integrada mas zona de almacenaje.El Estudio tiene pasillo ancho de entrada a zona diáfana sin dormitorio pero con cocina independiente y baño grande con ducha.Tienen preinstalación de aire acondicionado cada uno, radiadores eléctricos de energía azul. El agua caliente es por paneles solares centralizados, paredes en liso y suelos de gres, puertas en roble, techos de 2,65 lo que les da una gran amplitud. Son luminosos con sol de tarde con grandes ventanales de 2 hojas con rotura de puente térmico y climalit. Persianas térmicas. Puertas Blindadas.La finca es muy tranquila a 200 metros de Plaza de Barajas y del metro a 7 minutos, buenas calidades vivienda en entreplanta pero sin barreras arquitectónicas con ascensor inclus

In [1598]:
df_union.query("cod_barrio.isnull()")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal


In [1599]:
df_union.query("cod_barrio.notnull()")[["title","cod_barrio","cod_distrito","barrio","distrito","codigo_Postal"]]

,title,cod_barrio,cod_distrito,barrio,distrito,codigo_Postal
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",41.00,4.00,Recoletos,Salamanca,NaN
1,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid",213.00,21.00,Casco Histórico de Barajas,Barajas,NaN
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",24.00,2.00,Legazpi,Arganzuela,NaN
3,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",54.00,5.00,Hispanoamérica,Chamartín,NaN
4,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",54.00,5.00,Hispanoamérica,Chamartín,NaN
...,...,...,...,...,...,...
7869,Piso en venta en Calle de Dulce Chacón,162.00,16.00,Valdefuentes,Hortaleza,NaN
7870,Piso en venta en Calle de Marina Usera,126.00,12.00,Zofío,Usera,NaN
7871,Piso en venta en Zofío,126.00,12.00,Zofío,Usera,NaN
7872,Piso en venta en Calle de Raigras,126.00,12.00,Zofío,Usera,NaN


In [1600]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7874 entries, 0 to 7873
Data columns (total 71 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      7874 non-null   object 
 1   url                                        7874 non-null   object 
 2   precio                                     7874 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  7874 non-null   object 
 5   EUR/m2                                     7856 non-null   float64
 6   m2_constr                                  7861 non-null   float64
 7   distrito                                   7874 non-null   object 
 8   cod_distrito                               7874 non-null   float64
 9   barrio                                     7874 non-null   object 
 10  cod_barrio              

In [1601]:
dfbarrios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cod_distrito   132 non-null    int32 
 1   distrito       132 non-null    object
 2   cod_barrio     132 non-null    int32 
 3   cod_barrio_2   132 non-null    int64 
 4   barrio         132 non-null    object
 5   codigo_Postal  132 non-null    object
 6   distrito_n     132 non-null    object
 7   barrio_n       132 non-null    object
dtypes: int32(2), int64(1), object(5)
memory usage: 7.3+ KB


In [1602]:
df_union["cod_barrio"]

0       41.00
1      213.00
2       24.00
3       54.00
4       54.00
        ...  
7869   162.00
7870   126.00
7871   126.00
7872   126.00
7873   126.00
Name: cod_barrio, Length: 7874, dtype: float64

In [1603]:
df_union["barrio"] = df_union["barrio"].astype(object)
df_union["distrito"] = df_union["distrito"].astype(object)
df_union["cod_barrio"] = df_union["cod_barrio"].astype(int)
df_union["cod_distrito"] = df_union["cod_distrito"].astype(int)
df_union["codigo_Postal"] = df_union["codigo_Postal"].astype(object)

In [1604]:
#voy a asignar codigo postal teniendo cod_barrio
procesar_codpostal_nulos(df_union, dfbarrios)


41
28001
213
28042
24
28045
54
28016
54
28016
54
28016
54
28016
62
28020
51
28002
42
28001
72
28015
105
28024
105
28024
46
28006
14
28004
87
28035
54
28016
183
28031
132
28053
72
28015
41
28001
16
28013
74
28010
45
28006
51
28002
54
28016
54
28016
55
28016
45
28006
41
28001
41
28001
15
28015
42
28001
55
28016
73
28010
15
28015
43
28028
56
28046
56
28046
54
28016
35
28014
55
28016
16
28013
16
28013
41
28001
41
28001
41
28001
63
28020
51
28002
206
28037
11
28013
153
28027
55
28016
46
28006
55
28016
12
28012
74
28010
92
28008
62
28020
41
28001
41
28001
91
28011
52
28002
112
28019
15
28015
16
28013
41
28001
51
28002
75
28003
12
28012
74
28010
74
28010
41
28001
42
28001
15
28015
15
28015
207
28037
34
28009
32
28007
66
28039
164
28033
13
28014
62
28020
16
28013
51
28002
181
28031
41
28001
194
28032
194
28032
183
28031
15
28015
107
28044
127
28041
163
28043
44
28028
181
28031
42
28001
44
28028
15
28015
164
28033
72
28015
121
28041
12
28012
45
28006
12
28012
102
28011
152
28017
63
28020
34
280

In [1605]:
df_union.query("codigo_Postal.isna()")[["url","title","cod_barrio","cod_distrito","codigo_Postal"]]

,url,title,cod_barrio,cod_distrito,codigo_Postal


In [1606]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7874 entries, 0 to 7873
Data columns (total 71 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      7874 non-null   object 
 1   url                                        7874 non-null   object 
 2   precio                                     7874 non-null   object 
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  7874 non-null   object 
 5   EUR/m2                                     7856 non-null   float64
 6   m2_constr                                  7861 non-null   float64
 7   distrito                                   7874 non-null   object 
 8   cod_distrito                               7874 non-null   int32  
 9   barrio                                     7874 non-null   object 
 10  cod_barrio              

In [1607]:

df_union['precio'] = df_union['precio'].apply(lambda x: str(x).replace('.', '').strip() if isinstance(x, str) and float(x.replace('.', '')) >= 1000000 else x)

# Eliminar puntos de los separadores de miles y convertir a enteros
df_union['precio'] = df_union['precio'].apply(lambda x: int(str(x).replace('.', '').strip()))




In [1608]:


col_tiponuevo= ["precio"]

df_union[col_tiponuevo] = df_union[col_tiponuevo].astype("int")

In [1609]:
df_union["precio"].describe()

count        7874.00
mean      6079563.35
std       9327056.94
min             1.00
25%        950000.00
50%       2630000.00
75%       7890000.00
max     199500000.00
Name: precio, dtype: float64

In [1610]:
# elimino la vivienda con precio 1, es lo que aparece en la web

In [1611]:
# Filtrar filas donde el precio no es 1
df_union = df_union[df_union['precio'] != 1]

In [1612]:
df_union["precio"].describe()

count        7873.00
mean      6080335.55
std       9327397.62
min         25000.00
25%        950000.00
50%       2630000.00
75%       7890000.00
max     199500000.00
Name: precio, dtype: float64

In [1613]:
df_union.query("precio==25000")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal
1226,"Piso de tres habitaciones Calle del General Ricardos 46, San Isidro, Madrid",https://www.indomio.es/anuncios/96375609/,25000,NaN,nan,313.00,80.00,Carabanchel,11,San Isidro,113,San Isidro,calle del general ricardos 46,desconocido,nan,No indicado,Piso,NaN,Para reformar,Parcialmente amueblado,3,3.00,1,1.00,NaN,nan,0.00,1.00,nan,Individual,1960.0,6.00,NaN,NaN,NaN,NaN,NaN,1.00,104.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,100,NaN,norte,Indomio.com,2024-11-20,No indicado,NaN,"POR FAVOR , LEER ATENTAMENTE . VENTA 25% NUDA PROPIEDAD , resto del 50% mi madre y otro 25% hermana .25.000 € . Sin cargas . VEndo mi 25% de NUDA PROPIEDAD Usufructo de mi madre , persona 83 años NO visitable Perfecto inversión y constitución societariaContacto preferible Whatsapp",28019


In [1614]:
#elimino esta vivienda por que solo se vende el 25% de la misma
df_union = df_union[df_union['precio'] != 25000]

In [1615]:
df_union.query("precio==59000")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal
1570,"Casa unifamiliar Calle Rambla, Las Águilas, Madrid",https://www.indomio.es/anuncios/96538071/,59000,NaN,nan,472.00,125.00,Latina,10,Las Águilas,107,Las Águilas,calle rambla,desconocido,D,No indicado,Casa,NaN,nan,nan,nan,4.00,1,NaN,NaN,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,NaN,nan,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,Indomio.com,2024-11-20,Disponible,0.00,"Amplia casa en Murcia, perfecto para familias. La vivienda consta de 125 m construidos distribuidos en cuatro habitaciones, salón-comedor, cocina y un aseo. Localizada en el centro de Murcia cerca de Plaza de Las Flores, Real Casino de Murcia y Floridablanca Garden, en sus alrededores encontramos el University Hospital Reina Sofia, la Universidad de Murcia, supermercados, bancos, bibliotecas, farmacias, oficinas de correos, centros deportivos y varias paradas de autobús. Cuenta con fácil acceso a N-340. Oportunidad de inversión: inmueble procedente de ejecución hipotecaria sin posesión. Consulte con nuestro Agente Comercial la información adicional sobre el inmueble y las condiciones especiales de compraventa. El precio del inmueble ha sido fijado en atención a que el inmueble se adquiera por el comprador en estado de ocupado por lo que no aplicaría dicho precio si en el momento de formalización de la escritura pública el inmueble se encontrara libre de ocupantes. Imposibilidad de visita interior. Las fotografías pueden no corresponderse con el estado actual del inmueble.",28044
2369,"Piso de una habitación Pico Cejo, Numancia, Madrid",https://www.indomio.es/anuncios/96761591/,59000,NaN,nan,1.97,30.00,Puente de Vallecas,13,Numancia,136,Numancia,nan,desconocido,nan,No indicado,Piso,NaN,nan,nan,nan,1.00,1,NaN,0.00,nan,0.00,0.00,nan,NaN,1910.0,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,NaN,nan,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,0.00,"OPORTUNIDAD VENTA SIN POSESIÓN! El inmueble no se puede visitar interiormente. Este tipo de compra no admite hipoteca. LEA LAS CONDICIONES a continuación. Magnífica oportunidad de piso situado en Madrid, con una inmensa variedad de servicios, tanto de ocio como de necesidad, a su alrededor. La vivienda tiene una superficie construida de 30 m2 y útil 29 m2. Distribuido interiormente en diferentes dependencias y servicios. No dispone de ascensor en el edificio. LA FOTO NO IDENTIFICA LA PROPIEDAD EXACTA DE VENTA, SOLO INFORMA DE LA ZONA. CONDICIONES ESPECIALES DE COMPRA: En este tipo de adquisiciones se requieren una serie de requisitos detallados a continuación: 1º.- No permite hipoteca sobre el inmueble, los bancos no financian estas operaciones. 2º.- VENTA SIN POSESIÓN. LA PROPIEDAD NO SE PUEDE VER INTERIORMENTE, pero sí saber su ubicación exacta y aquellas características detalladas en este anuncio. (no disponemos de fotos de

In [1616]:
df_union["precio"].value_counts(dropna=False)

precio
2500000    50
1550000    39
2200000    39
1900000    38
8290000    38
           ..
445600      1
589000      1
169100      1
418000      1
4099900     1
Name: count, Length: 1823, dtype: int64

In [1617]:
df_union = df_union[df_union['url'] != 'https://www.indomio.es/anuncios/96761685/']

In [1618]:
df_union["tipologia"].value_counts(dropna=False)

tipologia
Piso           6647
Ático           330
Chalet          278
Casa            246
Dúplex          149
Apartamento     130
Estudio          73
Loft             18
Name: count, dtype: int64

In [1619]:
df_union.describe()

,precio,precio_anterior,EUR/m2,m2_constr,cod_distrito,cod_barrio,propiedad_completa,dormitorios,balcon,terraza,trastero,ascensor,plantas_edificio,inmueble_ingresos,alquiler_opcion_a_compra,nuda_propiedad,sup_comercial,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,fibra_optica,exterior,interior,interior_y_exterior,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,acceso_minusvalido,tiene_jardin
count,7871.00,55.00,7854.00,7859.00,7871.00,7871.00,113.00,7684.00,5855.00,6000.00,6297.00,6542.00,210.00,6.00,3.00,2.00,342.00,4282.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,7871.00,7871.00,3143.00,3143.00,3143.00,3143.00,3143.00,3143.00,253.00,1259.00
mean,6081870.51,830694.55,3719.56,197.73,7.32,74.29,1.00,2.89,0.14,0.32,0.15,0.51,5.51,1.00,1.00,1.00,271.82,0.42,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.09,0.01,0.00,0.01,0.03,0.01,0.02,0.45,0.26,0.32
std,9328085.54,977555.73,4009.57,632.69,5.61,55.99,0.00,1.46,0.35,0.47,0.36,0.50,2.61,0.00,0.00,0.00,227.90,0.49,0.03,0.07,0.04,0.06,0.02,0.03,0.05,0.06,0.07,0.12,0.11,0.28,0.11,0.05,0.07,0.17,0.11,0.15,0.50,0.44,0.47
min,59000.00,98100.00,0.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,950000.00,201500.00,6.90,70.00,3.00,34.00,1.00,2.00,0.00,0.00,0.00,0.00,4.00,1.00,1.00,1.00,108.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,2630000.00,440000.00,2980.50,104.00,6.00,55.00,1.00,3.00,0.00,0.00,0.00,1.00,5.00,1.00,1.00,1.00,204.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,7890000.00,955000.00,6347.25,167.00,11.00,113.00,1.00,3.00,0.00,1.00,0.00,1.00,7.00,1.00,1.00,1.00,429.75,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
max,199500000.00,4900000.00,27708.00,17324.00,21.00,215.00,1.00,24.00,1.00,1.00,1.00,1.00,23.00,1.00,1.00,1.00,946.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [1620]:
dfisolometros.query("url=='https://www.indomio.es/anuncios/91884849/'")

,url,m2_constr
1409,https://www.indomio.es/anuncios/91884849/,1775.00


In [1622]:
df_union.query("banos.isnull()")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal


In [1631]:
# Filtrar y actualizar valores
df_union[df_union['m2_constr'] < 13] = actualizar_valores(df_union[df_union['m2_constr'] < 13], dfisolometros, 'url', 'm2_constr')
df_union[df_union['m2_constr'] < 13] = actualizar_valores(df_union[df_union['m2_constr'] < 13], dfpcomlimpio, 'url', 'm2_constr')

C:\Users\extas\AppData\Local\Temp\ipykernel_7680\3684329844.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
C:\Users\extas\AppData\Local\Temp\ipykernel_7680\3684329844.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])


In [1632]:
df_union.query("m2_constr<15")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal
655,"Casa unifamiliar Ronda Sobradiel, Piovera, Madrid",https://www.indomio.es/anuncios/95678285/,3375000,NaN,nan,2.56,1.32,Hortaleza,16,Piovera,162,Piovera,nan,desconocido,nan,No indicado,Casa,NaN,Bueno / Habitable,nan,1,6.00,3+,NaN,1.00,nan,0.00,NaN,nan,NaN,1979.0,NaN,"Individual, frío/calor",Privado,NaN,NaN,NaN,NaN,2.82,nan,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,NaN,0.00,0,0,0.00,1,0.00,0.00,0.00,0.00,1.00,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,1.00,"Diplomatic Real Estate ofrece chalet de lujo con parcela arbolada con fuente, piscina, zonas verdes, cámaras de seguridad, alarma, habitación de seguridad. Plaza de garaje de 100 m2. Distribuido en; Planta principal: hall de entrada, despacho, 5 dormitorios y 5 baños, aseo invitados, sala de recepción, salón y comedor de más de 100 m2 y hall de entrada. Planta superior, salón diáfano, área de bar y baño. Sótano, garaje para 6 coches, gimnasio, despacho, bodega, baño, sala de música. desván de 148 m2, y trastero Esperamos su llamada en el 91 564 77 00. Diplomatic Real Estate con más de 25 años en el sector inmobiliario y una amplia cartera de viviendas de lujo, locales, oficinas y edificios ofrece asesoramiento completo a sus clientes tanto en alquiler como en compra venta de inmuebles. www.diplomatic. es",28055
1520,"Chalet 6 habitaciones, Valdemarín, Madrid",https://www.indomio.es/anuncios/96517461/,1800000,NaN,nan,1.55,1.16,Moncloa-Aravaca,9,Valdemarín,95,Valdemarín,nan,desconocido,nan,No indicado,Chalet,NaN,nan,nan,nan,6.00,3+,NaN,NaN,nan,0.00,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0,0.00,1,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,NaN,"Excepcional casa pareada con piscina en Pozuelo de AlarcónBARNES Madrid presenta esta casa pareada de 4 plantas con piscina, de 1.162 m² de parcela y 480 m² construidos de casa situada en la reconocida zona residencial de Pozuelo de Alarcón (Aravaca). Cuenta con 6 habitaciones (3 de ellas en suite con baño incorporado), 7 baños con ducha y 1 aseo de invitados. Además tiene 2 terrazas de grandes dimensiones con vistas al jardín.La finca tiene acceso por la puerta principal al amplio jardín y piscina.La casa es única debido a la reestructuración que se hizo en 2010 y 2015 de amplios espacios y excelente ubicación, manteniendo el estilo clásico con un toque moderno, y conservando el suelo de parquet original.La vivienda se distribuye de la siguiente manera:PLANTA PRINCIPAL:La entrada a la vivienda comunica desde un amplio vestíbulo al salón muy espacioso con unas increíbles vistas al jardín por un lado, y por el otro al comedor.Cuenta con unos amplios ventanales a través de los cuales entra muchísima luz natural con vistas al jardín, al porche y a la piscina.El comedor conecta con la cocina office de gran tamaño, totalmente equipada y amueblada, con zona de lavandería y un aseo de invitados. Ad

In [ ]:
# eliminar https://www.indomio.es/anuncios/96408507/
 #es un edificio


In [1510]:
df_union["banos"].value_counts(dropna='False')

banos
1.0      1553
2.0      1493
1        1082
2         923
3.0       741
         ... 
85.0        1
201.0       1
130.0       1
445.0       1
61.0        1
Name: count, Length: 118, dtype: int64

In [1513]:
df_union.query("banos.isnull()")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,gastos_comunidad,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,tiene_jardin,descripcion_larga,codigo_Postal
161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1512]:
df_union["banos"]=df_union["banos"].astype(int)

ValueError: invalid literal for int() with base 10: 'nan'

In [1511]:
df_union.query("banos>10")[["web"]]

TypeError: '>' not supported between instances of 'str' and 'int'

In [960]:
#Guardo la union de los dos datasets
df_union.to_csv("20241231_Union_Sil_Ale.csv",index=False)
df_union.to_feather("20241231_Union_Sil_Ale.ftr")